#  The Chinook database

Goal: [The Chinook database](https://github.com/lerocha/chinook-database) contains information about a fictional digital music shop - kind of like a mini-iTunes store. The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.

Method: The main analysis is based on exploring the dataset using SQL.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

A list of all tables and views in the Chinook database:

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Inside of the table 'customer':

In [3]:
%%sql
SELECT *
FROM customer
LIMIT 5;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


Inside of the table 'invoice':

In [4]:
%%sql
SELECT *
FROM invoice
LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


Inside of the table 'employee':

In [5]:
%%sql
SELECT *
FROM employee
LIMIT 5;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


## Task 1: Popular genres in the USA
write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store from the four being

|Artist Name|Genre| 
|-----------|-----------|
|Regal	| Hip-Hop |
| Red Tone	| Punk |
| Meteor and the Girls |	Pop |
| Slim Jim Bites |	Blues |

In [6]:
%%sql
WITH quantity_genre AS
(
SELECT 
    c.customer_id,
    c.country,
    i.invoice_id,
    il.track_id,
    il.quantity,
    t.genre_id,
    g.name
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
WHERE country = "USA"
),

quantity_genre_perc AS

(
SELECT 
    qg.name genre,
    SUM(qg.quantity) tracks_sold
    FROM quantity_genre qg
    GROUP BY 1
    ORDER BY 2 DESC
)
SELECT 
    qgp.genre,
    qgp.tracks_sold,
    ROUND(CAST(qgp.tracks_sold AS FLOAT)/ (
                    SELECT SUM(qgp.tracks_sold) 
                    FROM quantity_genre_perc qgp),3) percentage
    FROM quantity_genre_perc qgp;
    

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Pop,22,0.021
Latin,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


In the USA the most popular genres are Rock (by far being more popular accounting for 53% of the market), Alternative & Punk, and Metal which contribute to about 77% of the total USA market.

The results indicate that from the mentioned above list of artist the store should purcased:
- Red Tone	- Punk
- Slim Jim Bites	- Blues
- Meteor and the Girls - Pop

However, a better buy would be a rock album.

## Task 2: Best sales employee
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

We will write a query that finds the total dollar amount of sales assigned to each sales support agent within the company.

In [7]:
%%sql
WITH customer_employee AS
(
SELECT
             c.customer_id,
             c.support_rep_id,
             SUM(i.total) total_purchases
         FROM customer c
         INNER JOIN invoice i ON i.customer_id = c.customer_id
         GROUP BY 2
)
SELECT 
    e.first_name || " " || e.last_name employee,
    ROUND(ce.total_purchases, 0) total_sales,
    e.hire_date,
    (e.hire_date - e.birthdate) working_age
    FROM employee e
    INNER JOIN customer_employee ce ON ce.support_rep_id = e.employee_id
    ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


employee,total_sales,hire_date,working_age
Jane Peacock,1732.0,2017-04-01 00:00:00,44
Margaret Park,1584.0,2017-05-03 00:00:00,70
Steve Johnson,1394.0,2017-10-17 00:00:00,52


The most successful sales support agent is Jane Peacock, the youngest of three employees. She is followed by Margaret Park, the oldest of them, it is safe to assume that she is the most experienced of them all, and Steve Johnson, who was hired the last.

## Task 3: Analyzing Sales by Country
The task is to analyze the sales data for customers from each different country, using the country value from the customers table.

In particular the task is to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average order value

Because there are a number of countries with only one customer, we should group these customers as "Other" in our analysis.

In [8]:
%%sql
DROP VIEW IF EXISTS distinct_countries;

CREATE VIEW distinct_countries AS
     SELECT
         country,
         COUNT(customer_id) count
     FROM customer
     GROUP by 1
     ORDER by 2 DESC;


SELECT * FROM distinct_countries;

 * sqlite:///chinook.db
Done.
Done.
Done.


country,count
USA,13
Canada,8
France,5
Brazil,5
Germany,4
United Kingdom,3
Portugal,2
India,2
Czech Republic,2
Sweden,1


In [9]:
%%sql

WITH customer_total_country AS
(
     SELECT
        i.customer_id id,
        SUM(i.total) sum_total,
        c.country
     FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
     GROUP by 1
     ORDER by 2 DESC
),
country_sum_income_customers AS
(
SELECT 
    ctc.country,
    SUM(ctc.sum_total) sum_income,
    COUNT(ctc.id) sum_customers,
    CASE 
        WHEN COUNT(ctc.id) = 1 THEN "OTHER"
        ELSE ctc.country
    END AS country_category
    FROM customer_total_country ctc
    GROUP BY 1
    ORDER BY 2 DESC
)

SELECT 
    csic.country_category country,
    SUM(csic.sum_customers) customers,
    ROUND(SUM(csic.sum_income),2) total_sales,
    ROUND((CAST(SUM(csic.sum_income) AS FLOAT)/CAST(SUM(csic.sum_customers) AS FLOAT)),2) av_order_value
FROM country_sum_income_customers csic
GROUP BY 1 ORDER BY 3 DESC;

 * sqlite:///chinook.db
Done.


country,customers,total_sales,av_order_value
OTHER,15,1094.94,73.0
USA,13,1040.49,80.04
Canada,8,535.59,66.95
Brazil,5,427.68,85.54
France,5,389.07,77.81
Germany,4,334.62,83.66
Czech Republic,2,273.24,136.62
United Kingdom,3,245.52,81.84
Portugal,2,185.13,92.57
India,2,183.15,91.58


An alternative solutions is:

In [10]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        ROUND(SUM(unit_price),2) total_sales,
        ROUND(SUM(unit_price) / count(distinct customer_id),2) customer_lifetime_value,
        ROUND(SUM(unit_price) / count(distinct invoice_id),2) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.58
Other,15,1094.94,7.45,73.0


According to the results, the majority of customers are from the US, Canada, Brazil and France, and they generate the majority of the music store's revenue.
The Czech Republic has room for improvement as customers have made significant purchases.
Due to the small sample size, it is difficult to make any reliable claims, perhaps running cmapaign tests in each country will be useful before the store expands.

## Task 4: Albums vs Individual Tracks

 The task is to find out what percentage of purchases are individual tracks vs whole albums.
 According to the invoice_line every customer purchased only one item at the time.

In [11]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.